In [ ]:
from typing import *
import os

import librosa
import numpy as np
from numpy.fft import rfft
from numpy import pi
from matplotlib import pyplot as plt
from IPython.display import Audio
import cmath
import scipy
from scipy.signal import stft
import torch
from torch import nn
import torch.nn.functional as F

from cache_no_hash import cache
from blindDescend import blindDescend
from yin import yin
from harmonicSynth import HarmonicSynth, Harmonic

TWO_PI = np.pi * 2

In [ ]:
try:
    os.chdir('playground')
except FileNotFoundError:
    pass

In [ ]:
os.chdir('..')
from shared import *
from lobe import getLobe
from manual_fc import ManualFC
from hyper_params import HyperParams
from nitf import NITF
from dataset_definitions import DatasetDefinition
os.chdir('playground')

In [ ]:
def sino(freq, length):
    return np.sin(np.arange(length) * freq * TWO_PI / SR)

def playHard(data):
    return Audio(data, rate = SR)
def play(data, soft = .1):
    t = np.concatenate([data, [1]])
    length = round(soft * SR)
    t[:length ] = np.multiply(t[:length ], np.linspace(0, 1, length))
    t[-length:] = np.multiply(t[-length:], np.linspace(1, 0, length))
    return playHard(t)

def findPeaks(energy):
    slope = np.sign(energy[1:] - energy[:-1])
    extrema = slope[1:] - slope[:-1]
    return np.argpartition(
        (extrema == -2) * energy[1:-1], - N_HARMONICS,
    )[- N_HARMONICS:] + 1

def sft(signal, freq_bin):
    # Slow Fourier Transform
    return np.abs(np.sum(signal * np.exp(IMAGINARY_LADDER * freq_bin))) / PAGE_LEN

def refineGuess(guess, signal):
    def loss(x):
        if x < 0:
            return 0
        return - sft(signal, x)
    freq_bin, loss = blindDescend(loss, .01, .4, guess)
    return freq_bin * SR / PAGE_LEN, - loss

def widePlot(h = 3, w = 12):
    plt.gcf().set_size_inches(w, h)
    
def spectrum(signal, do_wide = True, trim = 130):
    energy = np.abs(rfft(signal * HANN)) / (PAGE_LEN / 2)
    X = np.linspace(0, SR / 2, len(energy))
    plt.plot(
        X     [:trim], 
        energy[:trim], 
    )
    plt.xlabel('freq (Hz)')
    if do_wide:
        widePlot()
    return energy

def spectrogram(signal, **kw):
    f, t, Zxx = stft(signal, fs=SR, **kw)
    plt.pcolormesh(t, f, np.abs(Zxx))

def concatSynth(synth, harmonics, n):
    buffer = []
    for i in range(n):
        synth.eat(harmonics)
        buffer.append(synth.mix())
    return np.concatenate(buffer)

def pitch2freq(pitch):
    return np.exp((pitch + 36.37631656229591) * 0.0577622650466621)

def freq2pitch(f):
    return np.log(f) * 17.312340490667562 - 36.37631656229591

def pagesOf(signal):
    for i in range(0, signal.size - PAGE_LEN + 1, PAGE_LEN):
        yield signal[i : i + PAGE_LEN]

def plotUnstretchedPartials(f0, n_partials = 14, color = 'r', alpha = .3):
    for i in range(1, n_partials + 1):
        freq = f0 * i
        plt.axvline(x = freq, color = color, alpha = alpha)


In [ ]:
timbre_points = [
    (700, .2), 
    (950, .05), 
    (2400, .05), 
    (2800, 0), 
    (3000, 0), 
]
timbre = ManualFC(
    torch.tensor([x[0] for x in timbre_points]), 
    torch.tensor([x[1] for x in timbre_points]), 
)
X = torch.linspace(0, NYQUIST, 1000)
plt.plot(X, timbre(X))

In [ ]:
f0 = 300
lobe = getLobe()
y = []
for i in range(N_HARMONICS):
    freq = f0 * (i + 1)
    mag = timbre(torch.tensor(freq)).numpy()
    y.append(sino(freq, SR) * mag)
y_long = np.stack(y).sum(axis=0)
y = y_long[:PAGE_LEN]
energy = np.abs(rfft(y * HANN)) / (PAGE_LEN / 2)
energy = torch.tensor(energy).float()
plt.plot(energy)
play(y_long)

In [ ]:
freqs = np.linspace(0, NYQUIST, SPECTRUM_SIZE)
freq_bin: float = freqs[1]
one_over_freq_bin = torch.tensor(1 / freq_bin).float()

In [ ]:
freqCube = torch.arange(0, SPECTRUM_SIZE).float()
freqCube = freqCube.unsqueeze(0).repeat(N_HARMONICS, 1)
freqCube.shape

In [ ]:
LADDER = torch.arange(0, N_HARMONICS).float().contiguous() + 1

In [ ]:
def forward(f0):
    freq = f0 * LADDER
    mag = timbre(freq)
    x = freqCube - (freq * one_over_freq_bin).unsqueeze(1)
    x = lobe(x)
    x = x * mag.unsqueeze(1)
    return x.sum(dim=0)

plt.plot(forward(f0), linewidth=4)
plt.plot(energy)

In [ ]:
from itertools import count
from time import sleep

In [ ]:
latent_f0 = torch.tensor(160.0, requires_grad=True)
optim = torch.optim.Adam([latent_f0], lr=1)
for epoch in range(10):
    spec_hat = forward(latent_f0)
    loss = F.mse_loss(spec_hat, energy)
    optim.zero_grad()
    loss.backward()
    optim.step()
    print(epoch, loss.item(), latent_f0.item())
    # sleep(.5)

In [ ]:
plt.plot(forward(latent_f0).detach(), linewidth=4)
plt.plot(energy)

Damn, we need multi-hot octave!

In [ ]:
PRIMES = (2,3,5,7,11,13)
DREDGE_RADIUS = 2
DREDGE_MULT = torch.ones((DREDGE_RADIUS * 2 + 1, )).float()
for i in range(DREDGE_RADIUS):
    DREDGE_MULT[DREDGE_RADIUS - i - 1] = 1 / PRIMES[i]
    DREDGE_MULT[DREDGE_RADIUS + i + 1] = PRIMES[i]
DREDGE_MULT

In [ ]:
class Dredge:
    def __init__(self, guess=200) -> None:
        self.freq = torch.tensor(
            float(freqNorm(guess)), 
            dtype=torch.float32, requires_grad=True, 
        )
        self.vector = torch.zeros(
            (DREDGE_RADIUS * 2 + 1, ), 
            dtype=torch.float32, 
        )
        self.vector[DREDGE_RADIUS] = 1
        self.vector.requires_grad = True
    
    def eval(self):
        return (DREDGE_MULT * freqDenorm(self.freq), self.vector)

    def simplify(self):
        old_vector = self.vector.detach()
        i = old_vector.argmax()
        if old_vector[i] / old_vector[DREDGE_RADIUS] < 2:
            return
        vector = torch.zeros(
            (DREDGE_RADIUS * 2 + 1, ), 
            dtype=torch.float32, 
        )
        vector[DREDGE_RADIUS] = old_vector[i]
        vector[DREDGE_RADIUS * 2 - i] = old_vector[DREDGE_RADIUS]
        vector.requires_grad = True
        self.vector = vector
        self.freq = freqNorm(freqDenorm(self.freq.detach()) * DREDGE_MULT[i])
        self.freq.requires_grad = True
    
    def parameters(self):
        return (self.freq, self.vector)

d = Dredge()
print(d.eval())
optim = torch.optim.Adam(d.parameters(), lr=1e-2)
for epoch in range(100):
    loss = (d.vector[4] - 0.8) ** 2 + (d.vector[3] - 0.1) ** 2
    optim.zero_grad()
    loss.backward()
    optim.step()
print(d.eval())
d.simplify()
print(d.eval())

In [ ]:
def trainDredge(dredge, ground_truth_energy, max_epoch=100, lr=1e-2):
    for epoch in range(max_epoch):
        if (epoch % 16 == 0):
            optim = torch.optim.Adam(dredge.parameters(), lr=lr)
            dredge.simplify()
        f0s, confidences = dredge.eval()
        spec_hat = []
        for i in range(f0s.shape[0]):
            spec_hat.append(forward(f0s[i]) * confidences[i])
        spec_hat = torch.stack(spec_hat, dim=0).sum(dim=0)
        loss = F.mse_loss(spec_hat, ground_truth_energy)
        optim.zero_grad()
        loss.backward()
        optim.step()
        yield epoch, loss, spec_hat

dredge = Dredge(450)
for epoch, loss, _ in trainDredge(dredge, energy):
    # print(epoch, loss.item())
    print(freqDenorm(dredge.freq).item())
    print(dredge.vector)
    # sleep(.3)

In [ ]:
# audio_name = 'dan'
audio_name = 'voice_scale'

y, sr = librosa.load(f'../dataset/{audio_name}.wav', sr=SR)
assert sr == SR
len(y)

In [ ]:
if audio_name == 'dan':
    seg = y[7400:8500]
else:
    # seg_start = round(.25 * SR)
    seg_start = round(.97 * SR)
    seg = y[seg_start:seg_start + PAGE_LEN * 4]
spectrogram(seg)
playHard(seg)

In [ ]:
energies = []
for page in pagesOf(seg):
    f0 = yin(page, SR, PAGE_LEN)
    print(f0)
    energy = spectrum(page, trim=60)
    energies.append(torch.tensor(energy).float())

In [ ]:
timbre_points = [
    (400, .015), 
    (450, .03), 
    (500, .03), 
    (550, 0.005), 
    (750, 0.002), 
    (800, .01), 
    (850, .01), 
    (950, 0), 
    (960, 0), 
]
timbre = ManualFC(
    torch.tensor([x[0] for x in timbre_points]), 
    torch.tensor([x[1] for x in timbre_points]) * 8 / 3, 
)
X = torch.linspace(0, 2500, 1000)
plt.plot(X, timbre(X))
widePlot()

In [ ]:
def forward(f0):
    freq = f0 * LADDER
    mag = timbre(freq)
    x = freqCube - (freq * one_over_freq_bin).unsqueeze(1)
    x = lobe(x)
    x = x * mag.unsqueeze(1)
    return x.sum(dim=0)

plt.plot(forward(f0)[:60], linewidth=4)
plt.plot(energy[:60])
widePlot()

In [ ]:
pages = [*pagesOf(seg)]
dredges = [Dredge(100) for _ in pages]
spec_hats = [None] * len(pages)
hist_freq = [[] for _ in pages]
hist_conf = [[] for _ in pages]

In [ ]:
def trainVoice(max_epoch=100):
    trainers = []
    for d, e in zip(dredges, energies):
        trainers.append(trainDredge(d, e, max_epoch, lr=2e-2))
    for epoch in range(max_epoch):
        print(epoch)
        for i, trainer in enumerate(trainers):
            _, loss, spec_hat = next(trainer)
            spec_hats[i] = spec_hat
            print(round(freqDenorm(dredges[i].freq.item())), end=', ')
            # print(d.vector)
            hist_freq[i].append(dredges[i].freq.detach().item())
            hist_conf[i].append(dredges[i].vector.detach().clone())
        print()

In [ ]:
dredges = [Dredge(100) for _ in pages]
spec_hats = [None] * len(pages)
hist_freq = [[] for _ in pages]
hist_conf = [[] for _ in pages]
# trainVoice(200)
trainVoice(10)

In [ ]:
i = 2
plt.plot([freqDenorm(x) for x in hist_freq[i]], 'o')
plt.plot([x[3] * 100 for x in hist_conf[i]])
widePlot()

In [ ]:
plt.plot(spec_hats[i].detach())
plt.plot(energies[i])
dredges[i].eval()
# dredges[i].vector.grad

for 3:2, it woudln't first / 3 then * 2. local min. 

In [ ]:
RATIOS = [3/2, 2, 3]
DREDGE_RADIUS = len(RATIOS)
DREDGE_MULT = torch.ones((DREDGE_RADIUS * 2 + 1, )).float()
for i in range(DREDGE_RADIUS):
    DREDGE_MULT[DREDGE_RADIUS - i - 1] = 1 / RATIOS[i]
    DREDGE_MULT[DREDGE_RADIUS + i + 1] = RATIOS[i]
DREDGE_MULT

it works. now add regularization

In [ ]:
def reguLoss(vector: torch.Tensor):
    loss = - vector.clip(max=0).sum()
    acc = torch.tensor(0).float()
    pos_v = vector.clip(min=0)
    de_pos_v = pos_v.detach()
    for i in range(vector.shape[0]):
        loss = loss + pos_v[i] * acc.clone()
        acc += de_pos_v[i]
    return loss

def trainDredge(dredge: Dredge, ground_truth_energy, max_epoch=100, lr=1e-2):
    for epoch in range(max_epoch):
        if (epoch % 16 == 0):
            optim = torch.optim.Adam(dredge.parameters(), lr=lr)
            dredge.simplify()
        f0s, confidences = dredge.eval()
        spec_hat = []
        for i in range(f0s.shape[0]):
            spec_hat.append(forward(f0s[i]) * confidences[i])
        spec_hat = torch.stack(spec_hat, dim=0).sum(dim=0)
        loss = F.mse_loss(spec_hat, ground_truth_energy)
        loss = loss + reguLoss(dredge.vector) * 1e-5
        optim.zero_grad()
        loss.backward()
        optim.step()
        yield epoch, loss, spec_hat


In [ ]:
dredges = [Dredge(100) for _ in pages]
spec_hats = [None] * len(pages)
hist_freq = [[] for _ in pages]
hist_conf = [[] for _ in pages]
# trainVoice(300)

In [ ]:
i = 2
plt.plot([freqDenorm(x) for x in hist_freq[i]], 'o')
plt.plot([x[3] * 100 for x in hist_conf[i]])
widePlot()

In [ ]:
for d in dredges:
    print(d.eval())

In [ ]:
from tqdm import tqdm

In [ ]:
def trajs(max_epoch=100, n_guesses=100):
    e = energies[0]
    trainers = []
    dredges: List[Dredge] = []
    for guess_f0 in np.linspace(
        pitch2freq(36), 
        pitch2freq(72), 
        n_guesses, 
    ):
        d = Dredge(guess_f0)
        dredges.append(d)
        trainers.append(trainDredge(d, e, max_epoch, lr=2e-2))
    for epoch in tqdm([*range(max_epoch)]):
        # print(epoch)
        for i, trainer in enumerate(trainers):
            _, loss, spec_hat = next(trainer)
            spec_hats[i] = spec_hat
            # print(d.vector)
            hist_freq[i].append(dredges[i].freq.detach().item())
            hist_conf[i].append(dredges[i].vector.detach().clone())
        # print()

In [ ]:
# n_guesses = 100
n_guesses = 20
spec_hats = [None] * n_guesses
hist_freq = [[] for _ in range(n_guesses)]
hist_conf = [[] for _ in range(n_guesses)]
# trajs(300, n_guesses)
trajs(3, n_guesses)

In [ ]:
for fs in hist_freq:
    plt.plot([freqDenorm(x) for x in fs], c='k', linewidth=.5)
    plt.plot([0, len(fs)], [freqDenorm(fs[0]), freqDenorm(fs[-1])], c='r', linewidth=.5)
plt.axhline(141, c='g', label='Truth')
plt.ylim(0, 700)
plt.legend()
plt.ylabel('F0 (Hz)')
plt.xlabel('Epoch')
plt.show()


Joint optim the timbre too

In [ ]:
template = HyperParams()
template.lr = None
template.weight_decay = 1e-9
template.optim_name = 'adam'
template.nif_width = 128
template.nif_depth = 6
template.n_vowel_dims = 2
template.nif_sees_f0 = False
template.nif_sees_amp = False
template.nif_sees_vowel = False
template.nif_abs_out = True
template.nif_abs_confidence = False
template.ground_truth_f0 = False
template.batch_size = 256
template.max_epoch = 600
hParams = template

In [ ]:
dDef = DatasetDefinition()
dDef.is_f0_latent = False
hParams.experiment_globals = {
    'datasetDef': dDef, 
    'dataset': None, 
}

In [ ]:
def viewEnv(nitf):
    with torch.no_grad():
        X = torch.linspace(
            pitch2freq(36), 
            NYQUIST, 
            1000, 
        )
        X = X.unsqueeze(1)
        Y = nitf.forward(freqNorm(X))
        plt.plot(X, Y, label='nitf')

In [ ]:
nitf = NITF(hParams, False)
viewEnv(nitf)
widePlot()
plt.show()

In [ ]:
def timbre(freq):
    return nitf(freqNorm(freq.unsqueeze(1)))[:, 0]
def trainDredge(dredge: Dredge, ground_truth_energy, max_epoch=100, lr=1e-2):
    for epoch in range(max_epoch):
        if (epoch % 16 == 0):
            optim = torch.optim.Adam([
                *dredge.parameters(), 
                *nitf.parameters(), 
            ], lr=lr)
            dredge.simplify()
        f0s, confidences = dredge.eval()
        confidences = confidences.abs()
        spec_hat = []
        for i in range(f0s.shape[0]):
            spec_hat.append(forward(f0s[i]) * confidences[i])
        spec_hat = torch.stack(spec_hat, dim=0).sum(dim=0)
        loss = F.mse_loss(spec_hat, ground_truth_energy)
        loss = loss + reguLoss(dredge.vector) * 1e-6
        yield epoch, loss, spec_hat
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            dredge.vector /= dredge.vector.norm()

def trainVoice(max_epoch=100):
    trainers = []
    for d, e in zip(dredges, energies):
        trainers.append(trainDredge(d, e, max_epoch, lr=1e-2))
    for epoch in range(max_epoch):
        # print(epoch)
        for i, trainer in enumerate(trainers):
            _, loss, spec_hat = next(trainer)
            with torch.no_grad():
                spec_hats[i] = spec_hat
                # print(round(freqDenorm(dredges[i].freq.item())), end=', ')
                # print(d.vector)
                hist_freq[i].append(dredges[i].freq.detach().item())
                hist_conf[i].append(dredges[i].vector.detach().clone())
                plt.cla()
                plt.plot(freqs, energies[i], label='truth')
                plt.plot(freqs, spec_hat, label='recon')
                viewEnv(nitf)
                F, C = dredges[i].eval()
                kw = dict(label='dredge')
                plt.vlines(
                    F, ymin=0, ymax=C.abs() * .02, colors='k', **kw, 
                )
                # for f, c in zip(F, C):
                #     plt.vlines(
                #         f, ymin=0, ymax=c.abs().item(), colors='k', **kw, 
                #     )
                #     kw.clear()
                plt.legend()
        yield
        # print()

In [ ]:
pages = pages[:1]
dredges = [Dredge(300) for _ in pages]
spec_hats = [None] * len(pages)
hist_freq = [[] for _ in pages]
hist_conf = [[] for _ in pages]
nitf = NITF(hParams, False)
tv = trainVoice(600)

In [ ]:
epoch = 0
for _ in tqdm([*range(50)]):
    next(tv)
    epoch += 1
widePlot()
plt.show()


In [ ]:
dredges[0].eval()
# DREDGE_MULT
print(dredges[0].vector)
dredges[0].vector.grad

In [ ]:
i = 0
plt.plot([freqDenorm(x) for x in hist_freq[i]], 'o')
plt.plot([x[0] * 100 for x in hist_conf[i]])
plt.plot([x[1] * 100 for x in hist_conf[i]])
plt.plot([x[2] * 100 for x in hist_conf[i]])
plt.plot([x[3] * 100 for x in hist_conf[i]])
plt.plot([x[4] * 100 for x in hist_conf[i]])
plt.plot([x[5] * 100 for x in hist_conf[i]])
plt.plot([x[6] * 100 for x in hist_conf[i]])
widePlot()

In [ ]:
def trainDredge(dredge: Dredge, ground_truth_energy, max_epoch=100, lr=1e-2):
    optimNitf = torch.optim.Adam([
        *nitf.parameters(), 
    ], lr=1e-3)
    for epoch in range(max_epoch):
        if (epoch % 16 == 0):
            optimFreq = torch.optim.Adam([
                dredge.freq, 
            ], lr=1e-3)
            optimVector = torch.optim.Adam([
                dredge.vector, 
            ], lr=1e-2)
            dredge.simplify()
        f0s, confidences = dredge.eval()
        confidences = confidences.abs()
        spec_hat = []
        for i in range(f0s.shape[0]):
            spec_hat.append(forward(f0s[i]) * confidences[i])
        spec_hat = torch.stack(spec_hat, dim=0).sum(dim=0)
        loss = F.mse_loss(spec_hat, ground_truth_energy)
        loss = loss + reguLoss(dredge.vector) * 1e-6
        yield epoch, loss, spec_hat
        optimNitf.zero_grad()
        optimFreq.zero_grad()
        optimVector.zero_grad()
        loss.backward()
        optimNitf.step()
        optimFreq.step()
        optimVector.step()
        with torch.no_grad():
            dredge.vector /= dredge.vector.norm()


In [ ]:
pages = pages[:1]
dredges = [Dredge(300) for _ in pages]
spec_hats = [None] * len(pages)
hist_freq = [[] for _ in pages]
hist_conf = [[] for _ in pages]
nitf = NITF(hParams, False)
N_EPOCHS = 1000
tv = trainVoice(N_EPOCHS)
epoch = 0

In [ ]:
for _ in tqdm([*range(
    N_EPOCHS, 
    # 10, 
)]):
    next(tv)
    plt.ylim(0, .04)
    plt.xlabel('Freq (Hz)')
    plt.title(f'epoch {epoch}')
    widePlot(6, 12)
    plt.savefig(f'./vid_out/a_{epoch}.png')
    epoch += 1
widePlot()
plt.show()
